In [1]:
import pandas as pd
import time 
import warnings
from tqdm.notebook import trange, tqdm
from os import listdir
from os.path import isfile, join
import glob

warnings.filterwarnings('ignore')

In [3]:
Target_KS_15 = pd.read_csv("КС-15P_in_out.csv",sep = ";")
Target_KS_15.head()

,Object,DateTime,Pin,Pout
0,2010,2021-01-01 00:00:00,"38,7",46
1,2010,2021-01-01 02:00:00,"38,9","46,2"
2,2010,2021-01-01 04:00:00,"38,9","46,4"
3,2010,2021-01-01 06:00:00,"39,1","46,5"
4,2010,2021-01-01 08:00:00,"39,3","46,6"


In [4]:
Target_KS_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13566 entries, 0 to 13565
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Object    13566 non-null  int64 
 1   DateTime  13566 non-null  object
 2   Pin       13566 non-null  object
 3   Pout      13566 non-null  object
dtypes: int64(1), object(3)
memory usage: 424.1+ KB


In [6]:
Target_KS_15["DateTime"] = pd.to_datetime(Target_KS_15["DateTime"])

In [9]:
Target_KS_15["dayofweek"] = Target_KS_15["DateTime"].dt.dayofweek

In [10]:
Target_KS_15.head()

,Object,DateTime,Pin,Pout,dayofweek
0,2010,2021-01-01 00:00:00,"38,7",46,4
1,2010,2021-01-01 02:00:00,"38,9","46,2",4
2,2010,2021-01-01 04:00:00,"38,9","46,4",4
3,2010,2021-01-01 06:00:00,"39,1","46,5",4
4,2010,2021-01-01 08:00:00,"39,3","46,6",4


In [11]:
vocab = {
    0 : 'Пн',
    1 : 'Вт',
    2 : 'Ср',
    3 : 'Чт',
    4 : 'Пт',
    5 : 'Сб',
    6 : 'Вс'
}


In [16]:
Target_KS_15["dayofweek"] = Target_KS_15["dayofweek"].replace(vocab, regex=True)

In [37]:
def create_target_set(target_name, how_to_shift):
    Target_KS = pd.read_csv(target_name+"P_in_out.csv",sep = ";")
    Target_KS["DateTime"] = pd.to_datetime(Target_KS_15["DateTime"])
    Target_KS["dayofweek"] = Target_KS["DateTime"].dt.dayofweek
    Target_KS["dayofweek"] = Target_KS["dayofweek"].replace(vocab, regex=True)
    Target_KS.columns = [f'{c}_'+ target_name if c!="DateTime" else c for c in Target_KS]
     #Тут будет заполнение пропусков чтобы дискретность была час в будущем
    if how_to_shift == "1":
                
        #Тут будет сдвиг когда-то в будущем
    else:
        return Target_KS
   
    

In [50]:
def prepare_dataset(target_name, how_to_shift_target):
    grs_list = glob.glob("ГРС/*")
    auto_plan_list = glob.glob("Автоплан/*")
    target_set = create_target_set(target_name, how_to_shift_target)
    schema_info = pd.read_csv(target_name+".csv",sep = ";")
    schema_info["DateTime"] = pd.to_datetime(schema_info["DateTime"])
    target_set = pd.merge(target_set,schema_info,how='left',on="DateTime")
    for grs in tqdm(grs_list):
        feature = pd.read_csv(grs, sep = ";")
        feature["DateTime"] = pd.to_datetime(feature["DateTime"])
        feature.columns = [f'{c}_'+ grs.split("\\")[1][:-4] if c!="DateTime" else c for c in feature]
        target_set = pd.merge(target_set,feature,how='left',on="DateTime")
    for auto_list in tqdm(auto_plan_list):
        feature = pd.read_csv(auto_list, sep = ";")
        if "Date" in feature.columns:
            feature["Data"] = feature["Date"]
            feature.drop(columns=["Date"],inplace=True)
        try:
            feature["DateTime"] = pd.to_datetime(feature["Data"])
        except KeyError:
            continue
        feature.drop(columns=["Data"],inplace=True)
        feature.columns = [f'{c}_'+ auto_list.split("\\")[1][:-4] if c!="DateTime" else c for c in feature]
        target_set = pd.merge(target_set,feature,how='left',on="DateTime")
    return target_set
        
    

In [51]:
t_set = prepare_dataset("КС-15", "1")

  0%|          | 0/119 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [52]:
t_set

,Object_КС-15,DateTime,Pin_КС-15,Pout_КС-15,dayofweek_КС-15,Object,Tnv,GPAinwork,PNA,Schema1,...,82_КС-19 Факт по фидерам,83_КС-19 Факт по фидерам,84_КС-19 Факт по фидерам,85_КС-19 Факт по фидерам,86_КС-19 Факт по фидерам,87_КС-19 Факт по фидерам,88_КС-19 Факт по фидерам,89_КС-19 Факт по фидерам,ID_Кто планировал,Login_Кто планировал
0,2010,2021-01-01 00:00:00,"38,7",46,Пт,2010.0,-9.0,2.0,-6,21.0,...,0,0,0,0,0,0,0,0,14.0,Немытых
1,2010,2021-01-01 02:00:00,"38,9","46,2",Пт,2010.0,-9.0,2.0,-6,21.0,...,0,0,0,0,0,0,0,0,NaN,NaN
2,2010,2021-01-01 04:00:00,"38,9","46,4",Пт,2010.0,-9.0,2.0,-6,21.0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,2010,2021-01-01 06:00:00,"39,1","46,5",Пт,2010.0,-10.0,2.0,-6,21.0,...,0,0,0,0,0,0,0,0,NaN,NaN
4,2010,2021-01-01 08:00:00,"39,3","46,6",Пт,2010.0,-10.0,2.0,-6,21.0,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13629,2010,2024-02-05 06:00:00,"47,3","47,3",Пн,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13630,2010,2024-02-05 08:00:00,"47,1","47,1",Пн,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13631,2010,2024-02-05 10:00:00,"46,9","46,9",Пн,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13632,2010,2024-02-05 12:00:00,"46,7","46,7",Пн,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
Target_KS = pd.read_csv("КС-15P_in_out.csv",sep = ";")
Target_KS["DateTime"] = pd.to_datetime(Target_KS_15["DateTime"])
Target_KS["dayofweek"] = Target_KS["DateTime"].dt.dayofweek

In [63]:
Target_KS

,Object,DateTime,Pin,Pout,dayofweek
0,2010,2021-01-01 00:00:00,"38,7",46,4
1,2010,2021-01-01 02:00:00,"38,9","46,2",4
2,2010,2021-01-01 04:00:00,"38,9","46,4",4
3,2010,2021-01-01 06:00:00,"39,1","46,5",4
4,2010,2021-01-01 08:00:00,"39,3","46,6",4
...,...,...,...,...,...
13561,2010,2024-02-05 06:00:00,"47,3","47,3",0
13562,2010,2024-02-05 08:00:00,"47,1","47,1",0
13563,2010,2024-02-05 10:00:00,"46,9","46,9",0
13564,2010,2024-02-05 12:00:00,"46,7","46,7",0


In [78]:
Target_KS.loc[Target_KS['dayofweek'] == 0,'DateTime'] - pd.DateOffset(hours=72)

36      2021-01-01 00:00:00
37      2021-01-01 02:00:00
38      2021-01-01 04:00:00
39      2021-01-01 06:00:00
40      2021-01-01 08:00:00
                ...        
13561   2024-02-02 06:00:00
13562   2024-02-02 08:00:00
13563   2024-02-02 10:00:00
13564   2024-02-02 12:00:00
13565   2024-02-02 14:00:00
Name: DateTime, Length: 1940, dtype: datetime64[ns]

In [80]:
Target_KS.where(Target_KS['dayofweek'] == 0)['DateTime'] - pd.DateOffset(hours=72)

0                       NaT
1                       NaT
2                       NaT
3                       NaT
4                       NaT
                ...        
13561   2024-02-02 06:00:00
13562   2024-02-02 08:00:00
13563   2024-02-02 10:00:00
13564   2024-02-02 12:00:00
13565   2024-02-02 14:00:00
Name: DateTime, Length: 13566, dtype: datetime64[ns]